# `onliner_init_script.sh`

In [1]:
!cat onliner_init_script.sh

#!/bin/bash

if [ "$1" == "--run-on-roger" ]; then
    INFRASTRUCTURE_DESCRIPTION_FILE="infrastructure_description_roger.json"
else
    INFRASTRUCTURE_DESCRIPTION_FILE="infrastructure_description_chameleon.json"
fi

python create_appliances.py $INFRASTRUCTURE_DESCRIPTION_FILE
python create_os_users.py $INFRASTRUCTURE_DESCRIPTION_FILE

python create_lc_operation.py

exit 0


In [2]:
#import argparse
import base64
import getpass
import json
import os
import re
import sys
import time
import uuid

from Crypto.PublicKey import RSA
import requests
from requests.auth import HTTPBasicAuth

In [3]:
CONFIG_FILE = 'infrastructure_description_chameleon.json'

RESERVATION_ID = '85f97018-f1a1-423e-81c6-2e71e7488fef'

## `create_appliances.py`

In [4]:
def create_appliances(configuration_file_path=None):
    target_host = "127.0.0.1"

    appliance_registry_url = "http://%s:8003" % (target_host)
    resource_manager_url = "http://%s:8002" % (target_host)

    image_name = "CENTOS-7_HADOOP"

#     configuration_file_path = None
#     if len(sys.argv) > 1:
#         configuration_file_path = sys.argv[1]

    if configuration_file_path is None:
        print("No configuration file passed as a parameter :-(")
        return 1

    with open(configuration_file_path) as data_file:
        infrastructures = json.load(data_file)["infrastructures"]
        for infrastructure in infrastructures:

            infrastructure_name = infrastructure["name"]
            infrastructure_url = infrastructure["contact_url"]
            infrastructure_type = infrastructure["type"]

            # Creation of site
            site_dict = {
                "name": infrastructure_name,
                "type": infrastructure_type,
                "contact_url": infrastructure_url
            }
            r = requests.post("%s/sites/" % (appliance_registry_url), json=site_dict,
                              auth=HTTPBasicAuth('admin', 'pass'))
            print(
                "- creation of site %s => %s %s" % (
                    infrastructure_name, r.status_code, r.json() if r.status_code >= 400 else ""))

            skip_actions_creation = False
            if len(sys.argv) > 1:
                if sys.argv[1] == "skip":
                    skip_actions_creation = True

            # If needed, create actions
            if not skip_actions_creation:
                actions = ["configure_node", "prepare_node", "update_master_node", "user_data", "update_hosts_file", "heat_template"]

                for action in actions:
                    # Creating a new action
                    action_dict = {
                        "name": action
                    }
                    r = requests.post("%s/actions/" % (appliance_registry_url), json=action_dict,
                                      auth=HTTPBasicAuth('admin', 'pass'))
                    print(
                        "- creation of action %s => %s %s" % (
                            action, r.status_code, r.json() if r.status_code >= 400 else ""))

            # Create appliances
            for dirname, dirnames, filenames in os.walk("appliances"):
                # print path to all subdirectories first.
                for subdirname in dirnames:
                    appliance_name = subdirname
                    complete_path = os.path.join(dirname, subdirname)
                    metadata_keys = ["description", "image", "image_name"]

                    # Collect appliance metadata
                    appliance_metadata = {
                        "description": "",
                        "image": "",
                        "image_name": ""
                    }
                    for metadata_key in metadata_keys:
                        metdata_file_address = "%s/%s.txt" % (complete_path, metadata_key)
                        if os.path.isfile(metdata_file_address):
                            with open(metdata_file_address) as f:
                                content = f.read()
                                appliance_metadata[metadata_key] = content

                    # Create a new appliance
                    appliance_dict = {
                        "name": appliance_name,
                        "logo_url": appliance_metadata["image"],
                        "description": appliance_metadata["description"]
                    }
                    r = requests.post("%s/appliances/" % (appliance_registry_url), json=appliance_dict,
                                      auth=HTTPBasicAuth('admin', 'pass'))
                    print("- creation of appliance %s => %s %s" % (
                        appliance_name, r.status_code, r.json() if r.status_code >= 400 else ""))

                    # Create an appliance implementation for the given site
                    sites = [infrastructure_name]

                    # Create an implementation of the appliance for each given site
                    for site in sites:
                        appliance_impl_name = "%s_%s" % (
                            appliance_name, site) if appliance_name != "common" else "common"
                        appliance_impl_logo_address = "%s/%s_image.txt" % (complete_path, appliance_impl_name)
                        # if os.path.isfile(appliance_impl_logo_address) or appliance_impl_name == "common":
                        appliance_impl_dict = {
                            "name": appliance_impl_name,
                            "appliance": appliance_name,
                            "image_name": appliance_metadata["image_name"] if appliance_name != "common" else "n.a.",
                            "site": site,
                            "logo_url": appliance_impl_logo_address
                        }
                        r = requests.post("%s/appliances_impl/" % (appliance_registry_url), json=appliance_impl_dict,
                                          auth=HTTPBasicAuth('admin', 'pass'))
                        print("  - creation of appliance_impl %s => %s %s" % (
                            appliance_impl_name, r.status_code, r.json() if r.status_code >= 400 else ""))

                        # Create an instance of each script for each appliance implementation
                        for script_dirname, script_dirnames, script_filenames in os.walk("%s" % (complete_path)):
                            for script_filename in script_filenames:
                                script_file_address = "%s/%s" % (complete_path, script_filename)
                                if not "heat_template.jinja2" in script_file_address:
                                    continue
                                with open(script_file_address) as script_f:
                                    action_name = re.sub(r'.*/', '', re.sub(r'.jinja2', '', script_file_address))
                                    print(action_name)
                                    script_content = script_f.read()
                                    script_dict = {
                                        "code": script_content,
                                        "appliance": appliance_impl_name if appliance_name != "common" else appliance_name,
                                        "action": action_name
                                    }
                                    r = requests.post("%s/scripts/" % (appliance_registry_url), json=script_dict,
                                                      auth=HTTPBasicAuth('admin', 'pass'))
                                    print("    - creation of script_impl %s => %s %s" % (
                                        action_name, r.status_code, r.json() if r.status_code >= 400 else ""))

In [5]:
create_appliances(CONFIG_FILE)

- creation of site chi@tacc => 201 
- creation of action configure_node => 201 
- creation of action prepare_node => 201 
- creation of action update_master_node => 201 
- creation of action user_data => 201 
- creation of action update_hosts_file => 201 
- creation of action heat_template => 201 
- creation of appliance common => 201 
  - creation of appliance_impl common => 201 
- creation of appliance hadoop => 201 
  - creation of appliance_impl hadoop_chi@tacc => 201 
heat_template
    - creation of script_impl heat_template => 201 
- creation of site kvm@roger => 201 
- creation of action configure_node => 400 {u'name': [u'action with this name already exists.']}
- creation of action prepare_node => 400 {u'name': [u'action with this name already exists.']}
- creation of action update_master_node => 400 {u'name': [u'action with this name already exists.']}
- creation of action user_data => 400 {u'name': [u'action with this name already exists.']}
- creation of action update_hosts_

## `create_os_users.py`

In [6]:
import datetime
from dateutil.tz import tzlocal

In [7]:
def create_os_users(configuration_file, host, port):
#     parser = argparse.ArgumentParser()

#     parser.add_argument('configuration_file', type=str)
#     parser.add_argument('-H', '--host', type=str, default='127.0.0.1',
#         help='Resource manager host')
#     parser.add_argument('-p', '--port', type=int, default=8002,
#         help='Resource manager port')

#     args = parser.parse_args()

    configuration_file_path = configuration_file
    target_host = host
    resource_manager_url = 'http://{}:{}'.format(target_host, port)

    with open(configuration_file_path) as data_file:
        credentials = json.load(data_file)["credentials"]

    for credential in credentials:
        print(credential)

        username = credential["username"]
        project_name = credential["project_name"]
        infrastructure_name = credential["infrastructure"]
        credential_name = credential["name"]

        # Creating a new user if needed
        user_dict = {
            "username": username,
            "password": "foo",
            "project": project_name
        }
        r = requests.post("%s/users/" % (resource_manager_url), json=user_dict, auth=HTTPBasicAuth('admin', 'pass'))

        user_id = 1

        # Get the key of the current user
        r = requests.get("%s/rsa_public_key/%s/" % (resource_manager_url, user_id),
                         auth=HTTPBasicAuth('admin', 'pass'))

        if r.status_code != 200:
            print("could not find the public key for user %s :(" % (user_id,))
            return 1

        public_key_str = r.json()["public_key"]
        print("(0) => %s (%s)" % (public_key_str, hash(public_key_str)))
        key = RSA.importKey(public_key_str)

        # Upload new credentials for the new user
        password = getpass.getpass("please provide an OpenStack password for (%s, %s)@%s:" % (
                                    username, project_name, infrastructure_name,))
        credentials = {
            "username": username,
            "password": password,
            "project": project_name
        }
        uncrypted_json_credentials = "%s" % (json.dumps(credentials))

        public_key = RSA.importKey(public_key_str)
        enc_data = public_key.encrypt(uncrypted_json_credentials, 32)

        crypted_json_credentials_b64 = base64.b64encode("%s" % (enc_data))

        # Upload the credentials to the resource_manager
        credentials_dict = {
            "credentials": crypted_json_credentials_b64,
            "site_name": infrastructure_name,
            "name": credential_name,
            "user": user_id
        }

        r = requests.post("%s/credentials/" % (resource_manager_url), json=credentials_dict,
                          auth=HTTPBasicAuth('admin', 'pass'))

        print(r.status_code)
        print(r.json())

In [8]:
create_os_users(CONFIG_FILE, '127.0.0.1', 8002)

{u'username': u'ntimkovi', u'flavor': u'baremetal', u'project_name': u'FG-392', u'name': u'chi@tacc_fg392', u'infrastructure': u'chi@tacc'}
(0) => -----BEGIN PUBLIC KEY-----
MIGfMA0GCSqGSIb3DQEBAQUAA4GNADCBiQKBgQCg45VQbrNVKuS4YDJHPj7W4VjV
cp6xKykhPe5OMkVuvlSH8dACu9WbMp1hZNWvhEhyrGTCs3feJP+clTyUVO6TLHgR
6GwXGZCyI3k359hy0kfk5RUvYX79zm0OjEbwX2oWNkdQUhy7HR7UTABFxCNs7WxU
Y1JD+OvDQIQdeEKvUQIDAQAB
-----END PUBLIC KEY----- (6966001895046344521)
please provide an OpenStack password for (ntimkovi, FG-392)@chi@tacc:········
201
{u'site_name': u'chi@tacc', u'name': u'chi@tacc_fg392', u'user': 1}


## `create_lc_operation.py`

In [9]:
def create_lc_operation():
    """Create an operation and run it, to demonstrate the architecture"""

    # target_host = "141.142.170.178"
    target_host = "127.0.0.1"

    operation_registry_url = "http://%s:8000" % (target_host)
    operation_manager_url = "http://%s:8001" % (target_host)
    resource_manager_url = "http://%s:8002" % (target_host)

    print("- Building the line_counter example")

    # Creation of an Operation
    operation_dict = {
        "name": "LineCounter",
        "description": "A simple line counter that can be used to demonstrate the complete architecture.",
        "string_parameters": """["env_var", "parameter"]""",
        "logo_url": "https://raw.githubusercontent.com/DIBBS-project/DIBBS-Architecture-Demo/master/misc/dibbs/linecounter.png",
        "file_parameters": """["input_file"]"""
    }
    print(" - creating of the line_counter operation")
    r = requests.post("%s/operations/" % (operation_registry_url), json=operation_dict,
                      auth=HTTPBasicAuth('admin', 'pass'))
    operation = r.json()
    operation_id = r.json().get("id", 1)
    if r.status_code < 300:
        print("   OK")
    else:
        print("   ERROR")

    # Implementing the Operation based on the hadoop appliance.
    implementation_dict = {
        "name": "line_counter_hadoop",
        "appliance": "hadoop",
        "operation": operation_id,
        "cwd": "~",
        "script": "export ENV_VAR=!{env_var} ; "
                  "curl https://raw.githubusercontent.com/DIBBS-project/DIBBS-Architecture-Demo/master/misc/archive.tgz > __archive.tar.gz ; "
                  "tar -xzf __archive.tar.gz ; "
                  "rm -f __archive.tar.gz ; "
                  "bash run_job.sh @{input_file} !{parameter} > stdout 2> stderr",
        "output_type": "file",
        "output_parameters": """{"file_path": "output.txt"}"""
    }
    print(" - implementing of the line_counter operation => %s")
    r = requests.post("%s/operationversions/" % (operation_registry_url), json=implementation_dict,
                      auth=HTTPBasicAuth('admin', 'pass'))
    implementation = r.json()
    if r.status_code < 300:
        print("   OK")
    else:
        print("   ERROR")

    # Creating an instance of the Operation
    instance_dict = {
        "name": "line_counter_instance",
        "process_definition_id": operation_id,
        "parameters": """{"env_var": "plop","parameter": "parameter"}""",
        "files": """{"input_file": "https://raw.githubusercontent.com/DIBBS-project/DIBBS-Architecture-Demo/master/misc/input.txt"}"""
    }
    print(" - creating an instance of the line_counter operation")
    r = requests.post("%s/instances/" % (operation_manager_url), json=instance_dict,
                      auth=HTTPBasicAuth('admin', 'pass'))
    instance = r.json()
    instance_id = instance.get("id", 1)
    if r.status_code < 300:
        print("   OK")
    else:
        print("   ERROR")

    # Get a token from the resource manager
    get_token_dict = {
        "username": "admin",
        "password": "pass"
    }
    r = requests.post("%s/api-token-auth/" % (resource_manager_url), json=get_token_dict,
                      auth=HTTPBasicAuth('admin', 'pass'))
    resource_manager_token = r.json().get("token", "")
    print(" - getting a token from the resource manager => %s" % (r.status_code))
    if r.status_code < 300:
        print("   OK")
    else:
        print("   ERROR")

    # Prepare an execution of the previously created instance
    execution_dict = {
        "operation_instance": instance_id,
        "callback_url": "http://plop.org",
        "force_spawn_cluster": "",
        "resource_provisioner_token": resource_manager_token ,
        "hints": """{{"credentials": ["chi@tacc_fg392"], "lease_id": "{}"}}""".format(RESERVATION_ID)
    }
    print(" - preparing an execution of the line_counter operation")
    r = requests.post("%s/executions/" % (operation_manager_url), json=execution_dict,
                      auth=HTTPBasicAuth('admin', 'pass'))
    execution = r.json()
    execution_id = execution.get("id", 0)
    if r.status_code < 300:
        print("   OK")
    else:
        print("   ERROR")

    # Launch the execution of the operation instance
    print(" - launching the execution of the line_counter operation => %s" % (r.status_code))
    r = requests.get("%s/exec/%s/run" % (operation_manager_url, execution_id),
                     auth=HTTPBasicAuth('admin', 'pass'))
    if r.status_code < 300:
        print("   OK")
    else:
        print("   ERROR")

    # Wait for the execution to finish
    print(" - Waiting for the execution to finish")

    execution_has_finished = False
    current_status = None
    previous_status = ""
    while not execution_has_finished:
        r = requests.get("%s/executions/%s" % (operation_manager_url, execution_id),
                         auth=HTTPBasicAuth('admin', 'pass'))

        data = r.json()
        current_status = data["status"]

        if current_status != previous_status:
            now = datetime.datetime.now(tz=tzlocal())
            print(" ({})  => {}".format(now.strftime('%H:%M:%S'), current_status))
            previous_status = current_status

        if current_status == "FINISHED":
            execution_has_finished = True

        if not execution_has_finished:
            time.sleep(2)

    # Download the output of the execution
    print(" - Download the output of the execution")
    r = requests.get("%s/executions/%s" % (operation_manager_url, execution_id),
                     auth=HTTPBasicAuth('admin', 'pass'))

    data = r.json()

    download_url = data["output_location"]
    if download_url is not None:
        r = requests.get(download_url, auth=HTTPBasicAuth('admin', 'pass'))

        # Write the downloaded file in a temporary file
        output_file_path = "/tmp/%s" % (uuid.uuid4())
        with open(output_file_path, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=128):
                fd.write(chunk)

        print("   => output has been downloaded in %s" % (output_file_path))

    return 0

In [10]:
import datetime
from dateutil.tz import tzlocal

In [12]:
create_lc_operation()

- Building the line_counter example
 - creating of the line_counter operation
   OK
 - implementing of the line_counter operation => %s
   OK
 - creating an instance of the line_counter operation
   OK
 - getting a token from the resource manager => 200
   OK
 - preparing an execution of the line_counter operation
   OK
 - launching the execution of the line_counter operation => 201
   OK
 - Waiting for the execution to finish
 (11:23:20)  => PENDING
 (11:23:26)  => DEPLOYING
 (11:30:55)  => DEPLOYED
 (11:31:06)  => PREPARING
 (11:35:48)  => RUNNING
 (11:36:35)  => COLLECTING
 (11:36:47)  => FINISHED
 - Download the output of the execution
   => output has been downloaded in /tmp/7a389f69-a40c-41b7-8d61-410a6fb6e41a


0

In [13]:
!cat /tmp/7a389f69-a40c-41b7-8d61-410a6fb6e41a


1	8

